In [ ]:
import tkinter as tk
from tkinter import messagebox, Menu
import random
import math

# ==========================================
# 1. KLASSEN UND HILFSFUNKTIONEN
# ==========================================

class Spielstein:
    """Repräsentiert ein Haus, Kirchenschiff oder Kirchturm."""
    def __init__(self, spieler_id, typ, ausrichtung='N'):
        self.spieler = spieler_id
        self.typ = typ             # 'Haus', 'Schiff', 'Turm'
        self.ausrichtung = ausrichtung

    def drehen(self):
        """Dreht den Stein nach einem Zug um 90 Grad im Uhrzeigersinn."""
        # Zyklus: N -> O -> S -> W -> N
        richtungen = ['N', 'O', 'S', 'W']
        if self.ausrichtung in richtungen:
            idx = richtungen.index(self.ausrichtung)
            self.ausrichtung = richtungen[(idx + 1) % 4]

    def get_farbe(self):
        return '#FF9999' if self.spieler == 1 else '#9999FF'

# --- KONSTANTEN ---
ROWS, COLS = 7, 7
CELL_SIZE = 90
WINDOW_SIZE = ROWS * CELL_SIZE
LINE_WIDTH = 3

OFFSET = 30

# ==========================================
# 2. POPUP FÜR RICHTUNGSWAHL
# ==========================================
class RichtungsWaehler(tk.Toplevel):
    def __init__(self, parent):
        super().__init__(parent)
        self.title("Ausrichtung wählen")
        self.geometry("250x200")
        self.ergebnis = None
        x = parent.winfo_x() + 50; y = parent.winfo_y() + 50
        self.geometry(f"+{x}+{y}")

        tk.Label(self, text="Wie soll der Stein stehen?", font=('Arial', 12)).pack(pady=20)
        btn_font = ('Arial', 10, 'bold')
        # Vertikal
        tk.Button(self, text="↑ Nord", command=lambda: self.setze('N')).pack(pady=5)
        tk.Button(self, text="↓ Süd", command=lambda: self.setze('S')).pack(pady=5)

        # Horizontal
        tk.Button(self, text="→ Ost", command=lambda: self.setze('O')).pack(pady=5)
        tk.Button(self, text="← West", command=lambda: self.setze('W')).pack(pady=5)

        self.transient(parent); self.grab_set(); self.wait_window(self)

    def setze(self, richtung):
        self.ergebnis = richtung
        self.destroy()

# ==========================================
# 3. HAUPTKLASSE SPIELBRETT
# ==========================================

class Spielbrett(tk.Canvas):

    def __init__(self, master):
        super().__init__(master, width=WINDOW_SIZE + 2 * OFFSET, height=WINDOW_SIZE + 2 * OFFSET, bg='#F0F0F0')
        self.pack()
        self.master = master
        self.ki_aktiv = False
        self.ki_spieler_id = 2   #Spieler 2 ist KI
        self.ki_schwierigkeit = "mittel"  # Standard: mittel
        self.spiel_aktiv = True  # Status ob Spiel läuft

        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]
        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1),
            2: self._initialisiere_steine(2)
        }
        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spieler_farben = {1: 'Rot', 2: 'Blau'}

        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld()
        self.bind("<Button-1>", self.handle_click)
        self.update_title()
        self.erstelle_menue()
        self.frage_spielmodus()
        
    def erstelle_menue(self):
        """Erstellt das Menü für das Spiel"""
        menubar = Menu(self.master)
        self.master.config(menu=menubar)
        
        # Spiel-Menü
        spiel_menu = Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Spiel", menu=spiel_menu)
        
        spiel_menu.add_command(label="Neues Spiel", command=self.neues_spiel)
        spiel_menu.add_separator()
        spiel_menu.add_command(label="Spielmodus wechseln", command=self.spielmodus_wechseln)
        spiel_menu.add_command(label="Schwierigkeitsgrad ändern", command=self.schwierigkeitsgrad_aendern)
        spiel_menu.add_separator()
        spiel_menu.add_command(label="Beenden", command=self.beenden)
        
        # Hilfe-Menü
        hilfe_menu = Menu(menubar, tearoff=0)
        menubar.add_cascade(label="Hilfe", menu=hilfe_menu)
        
        hilfe_menu.add_command(label="Spielregeln", command=self.zeige_spielregeln)
        hilfe_menu.add_command(label="Über", command=self.zeige_ueber)
    
    def neues_spiel(self):
        """Startet ein komplett neues Spiel"""
        if not self.spiel_aktiv:
            # Wenn Spiel schon vorbei war, einfach neu starten
            self.spiel_neustarten()
        else:
            # Frage nach Bestätigung, wenn Spiel läuft
            antwort = messagebox.askyesno("Neues Spiel", 
                "Möchtest du wirklich ein neues Spiel starten?\nDas aktuelle Spiel geht verloren.")
            if antwort:
                self.spiel_neustarten()
    
    def spiel_neustarten(self):
        """Setzt das Spiel komplett zurück"""
        # Zurücksetzen aller Variablen
        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]
        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1),
            2: self._initialisiere_steine(2)
        }
        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spiel_aktiv = True
        
        # Canvas leeren und neu zeichnen
        self.delete("all")
        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld()
        self.update_title()
        
        # KI-Einstellungen beibehalten, aber Spielmodus neu fragen
        self.frage_spielmodus()
    
    def spielmodus_wechseln(self):
        """Wechselt den Spielmodus (Mensch vs KI / Mensch vs Mensch)"""
        if self.spiel_aktiv:
            antwort = messagebox.askyesno("Spielmodus wechseln",
                "Möchtest du den Spielmodus wirklich wechseln?\nDas aktuelle Spiel wird neu gestartet.")
            if not antwort:
                return
        
        self.spiel_neustarten()
    
    def schwierigkeitsgrad_aendern(self):
        """Ändert den Schwierigkeitsgrad der KI"""
        if self.ki_aktiv:
            self.zeige_schwierigkeitsauswahl()
        else:
            messagebox.showinfo("Schwierigkeitsgrad",
                "Der Schwierigkeitsgrad kann nur geändert werden,\nwenn du gegen die KI spielst.")
    
    def beenden(self):
        """Beendet das Spiel"""
        if self.spiel_aktiv:
            antwort = messagebox.askyesno("Beenden", 
                "Möchtest du das Spiel wirklich beenden?")
            if antwort:
                self.master.quit()
        else:
            self.master.quit()
    
    def zeige_spielregeln(self):
        """Zeigt die Spielregeln an"""
        regeln = """
        LASS DIE KIRCHE IM DORF - Spielregeln
        
        ZIEL DES SPIELS:
        Als erster Spieler 9 eigene Steine in einer zusammenhängenden Gruppe zu haben,
        wobei mindestens ein Turm nicht am Rand steht und mindestens ein Kirchenschiff vorhanden ist.
        
        SPIELABLAUF:
        1. PLATZIERUNGSPHASE:
           - Jeder Spieler hat: 1 Turm, 1 Kirchenschiff, 7 Häuser
           - Türme und Kirchenschiffe müssen auf Ecken platziert werden
           - Häuser dürfen NICHT direkt orthogonal neben eigene Steine platziert werden
        
        2. BEWEGUNGSPHASE:
           - Steine gleiten in ihrer Ausrichtung (horizontal/vertikal)
           - Nach jedem Zug dreht sich der Stein um 90 Grad
        
        3. PFARRERTAUSCH:
           - Der Pfarrer (in der Mitte) kann mit einem blockierten Stein getauscht werden
           - Nur ein blockierter Stein (keine Zugmöglichkeit) kann getauscht werden
        
        STEIN-TYPEN:
        - Haus: Rechteck, bewegt sich horizontal/vertikal je nach Ausrichtung
        - Schiff: Kreis, bewegt sich horizontal/vertikal je nach Ausrichtung
        - Turm: Dreieck, bewegt sich horizontal/vertikal je nach Ausrichtung
        
        Viel Spaß beim Spielen!
        """
        messagebox.showinfo("Spielregeln", regeln)
    
    def zeige_ueber(self):
        """Zeigt Informationen über das Spiel an"""
        info = """
        LASS DIE KIRCHE IM DORF
        
        Ein Strategiespiel für 2 Spieler
        
        Version: 1.0
        Entwickelt mit Python und Tkinter
        
        KI-Schwierigkeitsgrade:
        - Einfach: Zufällige Züge
        - Mittel: Strategische Hausplatzierung
        - Schwer: Fortgeschrittene Strategie
        
        Steuerung:
        - Linksklick: Stein auswählen/platzieren
        - Menü: Spieloptionen
        
        © 2023
        """
        messagebox.showinfo("Über das Spiel", info)
        
    def frage_spielmodus(self):
        antwort = messagebox.askquestion("Spielmodus", "Möchtest du gegen die KI spielen?")
        if antwort == 'yes':
            self.ki_aktiv = True
            self.zeige_schwierigkeitsauswahl()
        else:
            self.ki_aktiv = False

    def zeige_schwierigkeitsauswahl(self):
        """Popup für Schwierigkeitsgrad-Auswahl"""
        popup = tk.Toplevel(self.master)
        popup.title("KI Schwierigkeitsgrad")
        popup.geometry("300x150")
        popup.transient(self.master)
        popup.grab_set()
        
        x = self.master.winfo_x() + 100
        y = self.master.winfo_y() + 100
        popup.geometry(f"+{x}+{y}")
        
        tk.Label(popup, text="Wähle den Schwierigkeitsgrad der KI:", 
                font=('Arial', 12)).pack(pady=10)
        
        # Variable für Radiobuttons
        self.ki_schwierigkeit_var = tk.StringVar(value=self.ki_schwierigkeit)
        
        tk.Radiobutton(popup, text="Einfach", variable=self.ki_schwierigkeit_var, 
                      value="einfach", font=('Arial', 10)).pack(anchor='w', padx=20)
        tk.Radiobutton(popup, text="Mittel", variable=self.ki_schwierigkeit_var, 
                      value="mittel", font=('Arial', 10)).pack(anchor='w', padx=20)
        tk.Radiobutton(popup, text="Schwer", variable=self.ki_schwierigkeit_var, 
                      value="schwer", font=('Arial', 10)).pack(anchor='w', padx=20)
        
        tk.Button(popup, text="OK", command=lambda: self.schwierigkeit_uebernehmen(popup), 
                 width=10).pack(pady=10)

    def schwierigkeit_uebernehmen(self, popup):
        self.ki_schwierigkeit = self.ki_schwierigkeit_var.get()
        popup.destroy()
        self.update_title()

    def _initialisiere_steine(self, spieler_id):
        steine = []
        steine.append(Spielstein(spieler_id, 'Turm'))
        steine.append(Spielstein(spieler_id, 'Schiff'))
        for _ in range(7): steine.append(Spielstein(spieler_id, 'Haus'))
        return steine

    def initialisiere_spielfeld(self):
        self.draw_pfarrer(self.pfarrer_pos[0], self.pfarrer_pos[1])

    # ---------------------------------------------------------
    # LOGIK-KERN: BERECHNUNG DER ERLAUBTEN ZÜGE (VEKTOR)
    # ---------------------------------------------------------

    def hat_eigene_orthogonale_nachbarn(self, row, col, spieler_id):
        nachbarn = [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]
        for r, c in nachbarn:
            if 1 <= r <= ROWS and 1 <= c <= COLS:
                stein = self.spielfeld_matrix[r-1][c-1]
                if stein is not None and stein.spieler == spieler_id:
                    return True
        return False

    def get_valid_moves(self, spieler_id):
        """Berechnet alle erlaubten Züge (BIDIREKTIONALES GLEITEN)."""
        moves = []
        vorrat = self.unplatzierte_steine[spieler_id]

        # --- PHASE 1: PLATZIEREN ---
        if len(vorrat) > 0:
            naechster_stein = vorrat[0]

            # A) Kirchenteile (Ecken, Diagonal-Check)
            if naechster_stein.typ in ['Turm', 'Schiff']:
                alle_ecken = [(1, 1), (1, 7), (7, 1), (7, 7)]
                partner_map = {(1, 1): (7, 7), (7, 7): (1, 1), (1, 7): (7, 1), (7, 1): (1, 7)}

                for (r, c) in alle_ecken:
                    if self.spielfeld_matrix[r-1][c-1] is None:
                        pr, pc = partner_map[(r, c)]
                        partner_stein = self.spielfeld_matrix[pr-1][pc-1]

                        diagonale_ok = True
                        if partner_stein is not None and partner_stein.spieler != spieler_id:
                            diagonale_ok = False

                        if diagonale_ok:
                            moves.append({'type': 'place', 'pos': (r, c), 'stone_type': naechster_stein.typ})

            # B) Häuser (Frei, keine eigenen Nachbarn)
            else:
                for r in range(1, ROWS + 1):
                    for c in range(1, COLS + 1):
                        if self.spielfeld_matrix[r-1][c-1] is None and (r, c) != self.pfarrer_pos:
                            if not self.hat_eigene_orthogonale_nachbarn(r, c, spieler_id):
                                moves.append({'type': 'place', 'pos': (r, c), 'stone_type': 'Haus'})

        # --- PHASE 2: BEWEGEN (Gleiten in beide Richtungen) ---
        else:
            for r in range(1, ROWS + 1):
                for c in range(1, COLS + 1):
                    stein = self.spielfeld_matrix[r-1][c-1]

                    if stein and stein.spieler == spieler_id:

                        # Bestimme die Achsen-Richtungen basierend auf der Ausrichtung
                        achsen_richtungen = []
                        if stein.ausrichtung in ['N', 'S']:
                            # Vertikale Achse: Oben (0, -1) UND Unten (0, 1)
                            achsen_richtungen = [(0, -1), (0, 1)]
                        else:
                            # Horizontale Achse: Rechts (1, 0) UND Links (-1, 0)
                            achsen_richtungen = [(1, 0), (-1, 0)]

                        # Prüfe BEIDE Richtungen der Achse
                        for dx, dy in achsen_richtungen:
                            distanz = 1
                            while True:
                                ziel_r = r + (dy * distanz)
                                ziel_c = c + (dx * distanz)

                                # 1. Ist Ziel noch auf dem Brett?
                                if not (1 <= ziel_r <= ROWS and 1 <= ziel_c <= COLS):
                                    break # Brettende

                                ziel_inhalt = self.spielfeld_matrix[ziel_r-1][ziel_c-1]

                                # 2. Blockiert durch Pfarrer?
                                if (ziel_r, ziel_c) == self.pfarrer_pos:
                                    break

                                # 3. Blockiert durch anderen Stein?
                                if ziel_inhalt is not None:
                                    break

                                # Feld frei -> Zug hinzufügen
                                moves.append({'type': 'move', 'from': (r, c), 'to': (ziel_r, ziel_c)})

                                # Weiter gleiten
                                distanz += 1

        return moves


    
    # --------------------------
    # SIEGBEDINGUNG
    # --------------------------
    def pruefe_sieg(self, spieler_id):
        """
        Prüft, ob der Spieler die Siegbedingung erfüllt:
        - Alle eigenen Steine zusammenhängend
        - Mindestens ein Turm nicht am Rand
        - Mindestens ein Kirchenschiff vorhanden
        """
        from collections import deque

        # Spielfeldgröße
        ROWS, COLS = len(self.spielfeld_matrix), len(self.spielfeld_matrix[0])

        # Alle eigenen Steine sammeln
        steine = [(r, c) for r in range(ROWS) for c in range(COLS)
              if self.spielfeld_matrix[r][c] and self.spielfeld_matrix[r][c].spieler == spieler_id]

        if not steine:
            return False

        # ----------------------
        # 1. Zusammenhängende Gruppe prüfen (BFS)
        # ----------------------
        besuchte = [[False]*COLS for _ in range(ROWS)]
        queue = deque()
        queue.append(steine[0])
        besuchte[steine[0][0]][steine[0][1]] = True
        count = 1

        while queue:
             r, c = queue.popleft()
             for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:  # orthogonal
                nr, nc = r + dr, c + dc
                if 0 <= nr < ROWS and 0 <= nc < COLS and not besuchte[nr][nc]:
                    s = self.spielfeld_matrix[nr][nc]
                    if s and s.spieler == spieler_id:
                        queue.append((nr, nc))
                        besuchte[nr][nc] = True
                        count += 1

                 # Mindestens 9 Steine müssen zusammenhängen
        if count < 9:
            return False

        # ----------------------
        # 2. Turm und Schiff prüfen
        # ----------------------
        turm_ok = False
        schiff_ok = False
        for r, c in steine:
            s = self.spielfeld_matrix[r][c]
            if s.typ == 'Turm':
                 # Turm darf nicht am Rand stehen
                if 0 < r < ROWS-1 and 0 < c < COLS-1:
                    turm_ok = True
            elif s.typ == 'Schiff':
                schiff_ok = True

        if not turm_ok or not schiff_ok:
            return False

        # ----------------------
        # Alles erfüllt -> Sieg
        # ----------------------
        return True

    # --------------------------
    #  Bedingungen für blokierten Zug
    # --------------------------
    def ist_blockierter_stein(self, row, col):
        """Prüft, ob der Stein an dieser Position blockiert ist (kein Zug möglich)."""
        stein = self.spielfeld_matrix[row-1][col-1]
        if not stein:
            return False
        valid_moves = self.get_valid_moves(stein.spieler)
        for m in valid_moves:
            if m['type'] == 'move' and m['from'] == (row, col):
                return False
        return True
    
     
    # --------------------------
    #   Pfarer Austaush
    # --------------------------

    def tausche_pfarrer_mit(self, row, col):
        """
        Tauscht den Pfarrer mit einem blockierten Stein.
        """
        if not self.ist_blockierter_stein(row, col):
            messagebox.showwarning("Ungültig", "Dieser Stein ist nicht blockiert!")
            return False

        stein = self.spielfeld_matrix[row-1][col-1]
        if stein is None:
            return False

        pf_r, pf_c = self.pfarrer_pos

        # Lösche Visualisierung
        self.delete(f"piece_{row}_{col}")
        self.delete(f"arrow_{row}_{col}")
        self.delete(f"label_{row}_{col}")
        self.delete("pfarrer")
        self.delete("pfarrer_label")

        # Tausch in Matrix
        self.spielfeld_matrix[pf_r-1][pf_c-1] = stein
        self.spielfeld_matrix[row-1][col-1] = None

        # Update Pfarrer Position
        self.pfarrer_pos = (row, col)

        # Visualisierung
        self.draw_piece(pf_r, pf_c, stein)
        self.draw_pfarrer(row, col)

    # ==========================================
    # KI-METHODEN FÜR VERSCHIEDENE SCHWIERIGKEITSGRADE
    # ==========================================

    def ki_macht_zug(self):
        """Hauptmethode für KI-Zug basierend auf Schwierigkeitsgrad"""
        if not self.spiel_aktiv:
            return
            
        valid_moves = self.get_valid_moves(self.ki_spieler_id)
        if not valid_moves: 
            return

        if self.ki_schwierigkeit == "einfach":
            self.ki_zug_einfach(valid_moves)
        elif self.ki_schwierigkeit == "mittel":
            self.ki_zug_mittel(valid_moves)
        else:  # schwer
            self.ki_zug_schwer(valid_moves)

    def ki_zug_einfach(self, valid_moves):
        """Einfache KI: Wählt zufälligen Zug"""
        zug = random.choice(valid_moves)
        
        if zug['type'] == 'place':
            r, c = zug['pos']
            stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
            # Zufällige Ausrichtung für einfache KI
            stein.ausrichtung = random.choice(['N', 'O', 'S', 'W'])
            self.platziere_stein(r, c, stein)
        else:
            self.ausgewaehlter_stein_pos = zug['from']
            self.fuehre_zug_aus(zug['to'][0], zug['to'][1])
        
        self.ausgewaehlter_stein_pos = None
        if self.pruefe_sieg(self.ki_spieler_id):
            self.spiel_beenden(f"KI ({self.ki_schwierigkeit}) hat gewonnen!")
        else:
            self.wechsle_spieler()

    def ki_zug_mittel(self, valid_moves):
        """Mittlere KI: Verwendet verbesserte strategische Heuristik für Häuser"""
        bewertete_zuege = []
        
        for zug in valid_moves:
            score = 0
            
            if zug['type'] == 'place':
                r, c = zug['pos']
                
                if zug['stone_type'] in ['Turm', 'Schiff']:
                    # Kirchenteile auf Ecken setzen
                    score += 25  # Höhere Priorität für Kirchenteile
                    
                    # Beste Ausrichtung für Bewegungsfreiheit finden
                    beste_richtung = 'N'
                    beste_richtung_score = -999
                    for richtung in ['N', 'O', 'S', 'W']:
                        richtung_score = self.strategische_bewerte_ausrichtung((r, c), richtung, self.ki_spieler_id)
                        if richtung_score > beste_richtung_score:
                            beste_richtung_score = richtung_score
                            beste_richtung = richtung
                    
                    score += beste_richtung_score
                    bewertete_zuege.append((score, zug, beste_richtung))
                else:
                    # STRATEGISCHE HAUS-PLATZIERUNG für mittlere KI
                    score += self.strategische_haus_platzierung((r, c), self.ki_spieler_id)
                    
                    # Blockiere Gegner, wenn möglich
                    score += self.blockiere_gegner((r, c), self.ki_spieler_id)
                    
                    # Finde beste Ausrichtung für zukünftige Bewegungen
                    beste_richtung = 'N'
                    beste_richtung_score = -999
                    for richtung in ['N', 'O', 'S', 'W']:
                        richtung_score = self.einfache_bewerte_ausrichtung((r, c), richtung)
                        if richtung_score > beste_richtung_score:
                            beste_richtung_score = richtung_score
                            beste_richtung = richtung
                    
                    score += beste_richtung_score
                    bewertete_zuege.append((score, zug, beste_richtung))
            else:
                # Bewegungen: Zielposition bewerten
                r, c = zug['to']
                score += self.strategische_position_bewertung(r, c, self.ki_spieler_id)
                bewertete_zuege.append((score, zug, None))
        
        # Besten Zug wählen
        bewertete_zuege.sort(key=lambda x: x[0], reverse=True)
        
        # Mittlere KI: Nimm einen der Top 2 Züge für etwas Variabilität
        if len(bewertete_zuege) > 2:
            top_zuege = bewertete_zuege[:2]
            # 80% Wahrscheinlichkeit für besten Zug, 20% für zweitbesten
            if random.random() < 0.8:
                bester_score, bester_zug, beste_richtung = top_zuege[0]
            else:
                bester_score, bester_zug, beste_richtung = top_zuege[1]
        else:
            bester_score, bester_zug, beste_richtung = bewertete_zuege[0] if bewertete_zuege else (0, None, None)
        
        if bester_zug:
            if bester_zug['type'] == 'place':
                r, c = bester_zug['pos']
                stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
                stein.ausrichtung = beste_richtung if beste_richtung else random.choice(['N', 'O', 'S', 'W'])
                self.platziere_stein(r, c, stein)
            else:
                self.ausgewaehlter_stein_pos = bester_zug['from']
                self.fuehre_zug_aus(bester_zug['to'][0], bester_zug['to'][1])
            
            self.ausgewaehlter_stein_pos = None
            if self.pruefe_sieg(self.ki_spieler_id):
                self.spiel_beenden(f"KI ({self.ki_schwierigkeit}) hat gewonnen!")
            else:
                self.wechsle_spieler()

    def ki_zug_schwer(self, valid_moves):
        """Schwere KI: Verwendet fortgeschrittene strategische Bewertungsfunktion"""
        bester_zug = None
        beste_ausrichtung = 'N'
        max_score = -999999

        for m in valid_moves:
            if m['type'] == 'place':
                r, c = m['pos']
                
                # Für Häuser: Erweiterte strategische Bewertung
                if m['stone_type'] == 'Haus':
                    for richtung in ['N', 'O', 'S', 'W']:
                        score = self.erweiterte_bewerte_haus_platzierung((r, c), richtung, self.ki_spieler_id)
                        if score > max_score:
                            max_score = score
                            bester_zug = m
                            beste_ausrichtung = richtung
                else:
                    # Kirchenteile
                    for richtung in ['N', 'O', 'S', 'W']:
                        score = (self.erweiterte_bewerte_kirchenteil((r, c), richtung, self.ki_spieler_id) +
                                self.bewerte_ausrichtung((r, c), richtung))
                        if score > max_score:
                            max_score = score
                            bester_zug = m
                            beste_ausrichtung = richtung
            else:
                # Bewegung bewerten
                score = self.erweiterte_bewerte_bewegung(m, self.ki_spieler_id)
                if score > max_score:
                    max_score = score
                    bester_zug = m
                    # Für Bewegungen: Ausrichtung ergibt sich aus Drehung

        if bester_zug:
            if bester_zug['type'] == 'place':
                r, c = bester_zug['pos']
                stein = self.unplatzierte_steine[self.ki_spieler_id].pop(0)
                stein.ausrichtung = beste_ausrichtung
                self.platziere_stein(r, c, stein)
            else:
                self.ausgewaehlter_stein_pos = bester_zug['from']
                self.fuehre_zug_aus(bester_zug['to'][0], bester_zug['to'][1])
            
            self.ausgewaehlter_stein_pos = None
            if self.pruefe_sieg(self.ki_spieler_id):
                self.spiel_beenden(f"KI ({self.ki_schwierigkeit}) hat gewonnen!")
            else:
                self.wechsle_spieler()

    def spiel_beenden(self, nachricht):
        """Beendet das Spiel mit einer Nachricht"""
        self.spiel_aktiv = False
        antwort = messagebox.askyesno("Spiel vorbei", f"{nachricht}\n\nMöchtest du ein neues Spiel starten?")
        if antwort:
            self.spiel_neustarten()
        else:
            # Titel aktualisieren, um anzuzeigen dass Spiel vorbei ist
            self.master.title(f"Lass die Kirche im Dorf - Spiel beendet")

    # ==========================================
    # VERBESSERTE BEWERTUNGSFUNKTIONEN FÜR STRATEGISCHE HAUS-PLATZIERUNG
    # ==========================================

    def strategische_haus_platzierung(self, pos, spieler_id):
        """Strategische Bewertung für Haus-Platzierung (für mittlere KI)"""
        r, c = pos
        score = 0
        
        # 1. DISTANZ ZU EIGENEN STEINEN (nicht direkt neben, aber strategisch nah)
        # Wir wollen Häuser in der Nähe von eigenen Steinen, aber nicht direkt daneben
        min_abstand = 999
        eigene_steine_pos = []
        
        # Finde alle eigenen Steine
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == spieler_id:
                    eigene_steine_pos.append((rr, cc))
                    dist = abs(rr - r) + abs(cc - c)
                    if dist < min_abstand and dist > 0:  # dist > 0 um direkte Nachbarn auszuschließen
                        min_abstand = dist
        
        # Optimale Distanz: 2-3 Felder entfernt (nicht direkt benachbart, aber nah genug für Verbindung)
        if 2 <= min_abstand <= 3:
            score += 15  # Perfekte Distanz für spätere Verbindung
        elif min_abstand == 1:
            score -= 20  # Direkter Nachbar ist nicht erlaubt (aber wird sowieso verhindert)
        elif min_abstand == 4:
            score += 5   # Etwas zu weit, aber akzeptabel
        elif min_abstand >= 5:
            score -= 10  # Zu weit weg von eigenen Steinen
        
        # 2. POTENZIELLE VERBINDUNGSMÖGLICHKEITEN
        # Zähle diagonale Nachbarn (diese sind für Verbindungen wichtig)
        diagonale_nachbarn = [(r-1, c-1), (r-1, c+1), (r+1, c-1), (r+1, c+1)]
        diagonale_eigene = 0
        for dr, dc in diagonale_nachbarn:
            if 1 <= dr <= ROWS and 1 <= dc <= COLS:
                stein = self.spielfeld_matrix[dr-1][dc-1]
                if stein and stein.spieler == spieler_id:
                    diagonale_eigene += 1
        
        if diagonale_eigene >= 2:
            score += 20  # Exzellente Verbindungsmöglichkeiten
        elif diagonale_eigene == 1:
            score += 8   # Gute Verbindungsmöglichkeit
        else:
            score -= 5   # Keine direkten Verbindungsmöglichkeiten
        
        # 3. BLOCKIERE GEGNER
        gegner_id = 1 if spieler_id == 2 else 2
        diagonale_gegner = 0
        for dr, dc in diagonale_nachbarn:
            if 1 <= dr <= ROWS and 1 <= dc <= COLS:
                stein = self.spielfeld_matrix[dr-1][dc-1]
                if stein and stein.spieler == gegner_id:
                    diagonale_gegner += 1
        
        if diagonale_gegner >= 2:
            score += 18  # Blockiert Gegner effektiv
        elif diagonale_gegner == 1:
            score += 6   # Leichte Blockade
        
        # 4. ZENTRUM BEVORZUGEN (für bessere Bewegungsmöglichkeiten)
        dist_zentrum = abs(r - 4) + abs(c - 4)
        score += max(0, 8 - dist_zentrum)  # Je näher am Zentrum, desto besser
        
        # 5. RAND MEIDEN (außer für strategische Zwecke)
        if r == 1 or r == ROWS or c == 1 or c == COLS:
            score -= 12
        
        return score

    def strategische_bewerte_ausrichtung(self, pos, richtung, spieler_id):
        """Strategische Ausrichtungsbewertung für Kirchenteile"""
        r, c = pos
        score = 0
        
        # Bewegungsfreiheit in dieser Ausrichtung
        achsen_vektoren = [(0, -1), (0, 1)] if richtung in ['N', 'S'] else [(1, 0), (-1, 0)]
        
        for dx, dy in achsen_vektoren:
            freie_felder = 0
            for dist in range(1, 4):  # Betrachte bis zu 3 Felder in jede Richtung
                nr, nc = r + (dy * dist), c + (dx * dist)
                if not (1 <= nr <= ROWS and 1 <= nc <= COLS):
                    break
                if self.spielfeld_matrix[nr-1][nc-1] or (nr, nc) == self.pfarrer_pos:
                    break
                freie_felder += 1
            score += freie_felder * 3
        
        # Strategische Überlegung: Ausrichtung weg von Gegnersteinen
        gegner_id = 1 if spieler_id == 2 else 2
        for dx, dy in achsen_vektoren:
            for dist in range(1, 3):
                nr, nc = r + (dy * dist), c + (dx * dist)
                if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                    stein = self.spielfeld_matrix[nr-1][nc-1]
                    if stein and stein.spieler == gegner_id:
                        score -= 10  # Ausrichtung auf Gegner ist schlecht
                        break
        
        return score

    def blockiere_gegner(self, pos, spieler_id):
        """Bewertet, wie gut diese Position den Gegner blockiert"""
        r, c = pos
        score = 0
        gegner_id = 1 if spieler_id == 2 else 2
        
        # Schau nach Häuser-Clustern des Gegners
        gegner_haeuser = []
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == gegner_id and stein.typ == 'Haus':
                    gegner_haeuser.append((rr, cc))
        
        # Wenn Gegner schon mehrere Häuser hat, versuche sie zu blockieren
        if len(gegner_haeuser) >= 2:
            # Finde Cluster von Gegnerhäusern
            for hr, hc in gegner_haeuser:
                dist = abs(hr - r) + abs(hc - c)
                if 2 <= dist <= 3:  # Strategische Blockadeposition
                    score += 15
                elif dist == 1:
                    score += 25  # Direkte Blockade (wenn möglich)
        
        return score

    def strategische_position_bewertung(self, r, c, spieler_id):
        """Strategische Positionsbewertung für Bewegungen"""
        score = 0
        
        # 1. Verbindung zu eigenen Steinen
        eigene_nachbarn = 0
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = r + dr, c + dc
            if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                stein = self.spielfeld_matrix[nr-1][nc-1]
                if stein and stein.spieler == spieler_id:
                    eigene_nachbarn += 1
        
        if eigene_nachbarn >= 2:
            score += 25  # Starke Verbindung
        elif eigene_nachbarn == 1:
            score += 10  # Gute Verbindung
        
        # 2. Distanz zum Zentrum
        dist_zentrum = abs(r - 4) + abs(c - 4)
        score += max(0, 10 - dist_zentrum * 2)
        
        # 3. Blockiere Gegner
        gegner_id = 1 if spieler_id == 2 else 2
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = r + dr, c + dc
            if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                stein = self.spielfeld_matrix[nr-1][nc-1]
                if stein and stein.spieler == gegner_id:
                    score += 8  # Blockade von Gegner
        
        return score

    # ==========================================
    # ERWEITERTE BEWERTUNGSFUNKTIONEN FÜR SCHWERE KI
    # ==========================================

    def erweiterte_bewerte_haus_platzierung(self, pos, richtung, spieler_id):
        """Erweiterte strategische Bewertung für Haus-Platzierung (für schwere KI)"""
        r, c = pos
        score = 0
        
        # 1. CLUSTER-BILDUNG (sehr wichtig für Sieg)
        score += self.bewerte_cluster_potenzial(r, c, spieler_id) * 3
        
        # 2. BLOCKADE DES GEGNERS
        score += self.bewerte_gegner_blockade(r, c, spieler_id) * 2
        
        # 3. BEWEGUNGSFREIHEIT (für spätere Bewegungsphase)
        score += self.bewerte_bewegungsfreiheit(r, c, richtung) * 1.5
        
        # 4. STRATEGISCHE POSITION (Zentrum, wichtige Linien)
        score += self.bewerte_strategische_position(r, c, spieler_id)
        
        # 5. SIEGBEDINGUNGEN VORBEREITEN
        score += self.bewerte_sieg_vorbereitung(r, c, spieler_id) * 2
        
        return score

    def erweiterte_bewerte_kirchenteil(self, pos, richtung, spieler_id):
        """Erweiterte Bewertung für Turm/Schiff"""
        r, c = pos
        score = 50  # Basiswert für Kirchenteile
        
        # Ecken sind gut für Kirchenteile
        if (r, c) in [(1,1), (1,7), (7,1), (7,7)]:
            score += 20
        
        # Bewegungsfreiheit für dieses Teil
        score += self.bewerte_bewegungsfreiheit(r, c, richtung) * 2
        
        # Schutz vor Blockade
        score += self.bewerte_schutz_position(r, c, spieler_id)
        
        return score

    def erweiterte_bewerte_bewegung(self, zug, spieler_id):
        """Erweiterte Bewertung für Bewegung"""
        from_r, from_c = zug['from']
        to_r, to_c = zug['to']
        score = 0
        
        # 1. Verbesserte Position?
        score += (self.bewerte_position(to_r, to_c, spieler_id) - 
                 self.bewerte_position(from_r, from_c, spieler_id)) * 2
        
        # 2. Blockiere ich den Gegner mit dieser Bewegung?
        score += self.bewerte_gegner_blockade_nach_bewegung(zug, spieler_id)
        
        # 3. Entblockiere ich mich selbst?
        if self.ist_blockierter_stein(from_r, from_c):
            score += 30  # Sehr gut, blockierten Stein bewegen
        
        # 4. Komme ich meinen eigenen Steinen näher?
        score += self.bewerte_naehe_zu_eigenen(to_r, to_c, spieler_id) * 1.5
        
        return score

    def bewerte_cluster_potenzial(self, r, c, spieler_id):
        """Bewertet das Potenzial, einen Cluster von Häusern zu bilden"""
        score = 0
        eigene_steine = []
        
        # Sammle alle eigenen Steine
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == spieler_id:
                    eigene_steine.append((rr, cc))
        
        # Finde den nächstgelegenen eigenen Stein
        if eigene_steine:
            min_dist = min(abs(r - sr) + abs(c - sc) for sr, sc in eigene_steine)
            # Optimale Distanz für Cluster-Bildung: 2 Felder (diagonal oder über ein Feld)
            if min_dist == 2:
                score += 25  # Perfekt für Cluster
            elif min_dist == 3:
                score += 15  # Gut für Cluster
            elif min_dist == 1:
                score -= 10  # Zu nah (aber wird durch Regeln verhindert)
            else:
                score += max(0, 10 - min_dist)  # Je näher, desto besser
        else:
            score += 5  # Erster Stein
        
        # Zähle diagonale Nachbarn für Cluster-Bildung
        diagonale = [(r-1,c-1), (r-1,c+1), (r+1,c-1), (r+1,c+1)]
        diagonale_eigene = 0
        for dr, dc in diagonale:
            if 1 <= dr <= ROWS and 1 <= dc <= COLS:
                stein = self.spielfeld_matrix[dr-1][dc-1]
                if stein and stein.spieler == spieler_id:
                    diagonale_eigene += 1
        
        score += diagonale_eigene * 12  # Diagonale Nachbarn sind exzellent für Cluster
        
        return score

    def bewerte_gegner_blockade(self, r, c, spieler_id):
        """Bewertet, wie gut diese Position den Gegner blockiert"""
        gegner_id = 1 if spieler_id == 2 else 2
        score = 0
        
        # Finde alle Gegnersteine
        gegner_steine = []
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == gegner_id:
                    gegner_steine.append((rr, cc))
        
        # Blockiere potenzielle Bewegungsrichtungen des Gegners
        for gr, gc in gegner_steine:
            # Wenn Gegnerstein in der Nähe ist
            dist = abs(gr - r) + abs(gc - c)
            if 1 <= dist <= 3:
                # Prüfe, ob wir eine wichtige Linie blockieren
                # Horizontale oder vertikale Ausrichtung des Gegners?
                if dist == 2 and (gr == r or gc == c):
                    score += 20  # Direkte Blockade einer Bewegungsachse
                elif dist == 1:
                    score += 30  # Sehr nahe Blockade
                else:
                    score += 10  # Allgemeine Blockade
        
        return score

    def bewerte_bewegungsfreiheit(self, r, c, richtung):
        """Bewertet die Bewegungsfreiheit in gegebener Ausrichtung"""
        score = 0
        achsen_vektoren = [(0, -1), (0, 1)] if richtung in ['N', 'S'] else [(1, 0), (-1, 0)]
        
        for dx, dy in achsen_vektoren:
            freie_felder = 0
            for dist in range(1, 7):  # Bis zum Brettrand
                nr, nc = r + (dy * dist), c + (dx * dist)
                if not (1 <= nr <= ROWS and 1 <= nc <= COLS):
                    break
                if self.spielfeld_matrix[nr-1][nc-1] or (nr, nc) == self.pfarrer_pos:
                    break
                freie_felder += 1
            # Quadratische Bewertung: Mehr freie Felder sind exponentiell besser
            score += freie_felder * freie_felder
        
        return score

    def bewerte_strategische_position(self, r, c, spieler_id):
        """Bewertet die strategische Position auf dem Brett"""
        score = 0
        
        # Zentrum ist wichtig für Bewegungsfreiheit
        center_r, center_c = 4, 4
        dist_to_center = abs(r - center_r) + abs(c - center_c)
        score += max(0, 15 - dist_to_center * 2)
        
        # Positionen nahe dem Pfarrer sind strategisch wertvoll
        pfarrer_r, pfarrer_c = self.pfarrer_pos
        dist_to_pfarrer = abs(r - pfarrer_r) + abs(c - pfarrer_c)
        if 2 <= dist_to_pfarrer <= 3:
            score += 12  # Gute Position für Pfarrertausch
        
        # Kontrolle wichtiger Linien
        if r == 4:  # Mittlere horizontale Linie
            score += 8
        if c == 4:  # Mittlere vertikale Linie
            score += 8
        
        return score

    def bewerte_sieg_vorbereitung(self, r, c, spieler_id):
        """Bewertet, wie gut diese Position auf Siegbedingungen vorbereitet"""
        score = 0
        
        # Für den Sieg müssen 9 Steine zusammenhängen
        # Diese Position sollte helfen, eine große zusammenhängende Gruppe zu bilden
        
        # Zähle, wie viele eigene Steine in einem 3x3 Bereich um diese Position sind
        eigene_in_umgebung = 0
        for dr in range(-2, 3):
            for dc in range(-2, 3):
                nr, nc = r + dr, c + dc
                if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                    stein = self.spielfeld_matrix[nr-1][nc-1]
                    if stein and stein.spieler == spieler_id:
                        eigene_in_umgebung += 1
        
        score += eigene_in_umgebung * 4
        
        # Position sollte nicht isoliert sein
        if eigene_in_umgebung == 0:
            score -= 20
        
        return score

    def bewerte_position(self, r, c, spieler_id):
        """Allgemeine Positionsbewertung"""
        score = 0
        
        # Nähe zu eigenen Steinen
        score += self.bewerte_naehe_zu_eigenen(r, c, spieler_id) * 2
        
        # Distanz zum Zentrum
        dist_zentrum = abs(r - 4) + abs(c - 4)
        score += max(0, 10 - dist_zentrum)
        
        # Nicht am Rand
        if not (r == 1 or r == ROWS or c == 1 or c == COLS):
            score += 5
        
        return score

    def bewerte_naehe_zu_eigenen(self, r, c, spieler_id):
        """Bewertet Nähe zu eigenen Steinen"""
        score = 0
        min_dist = 999
        
        for rr in range(1, ROWS + 1):
            for cc in range(1, COLS + 1):
                stein = self.spielfeld_matrix[rr-1][cc-1]
                if stein and stein.spieler == spieler_id:
                    dist = abs(rr - r) + abs(cc - c)
                    if dist < min_dist and dist > 0:
                        min_dist = dist
        
        if min_dist < 999:
            # Optimale Distanz: 1-3 Felder
            if 1 <= min_dist <= 3:
                score += 15 - min_dist * 2  # Je näher, desto besser (aber > 0)
        
        return score

    def bewerte_schutz_position(self, r, c, spieler_id):
        """Bewertet, wie geschützt eine Position ist"""
        score = 0
        
        # Kirchenteile sollten nicht zu exponiert sein
        # Zähle eigene Steine in der Nähe zum Schutz
        eigene_nah = 0
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                if dr == 0 and dc == 0:
                    continue
                nr, nc = r + dr, c + dc
                if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                    stein = self.spielfeld_matrix[nr-1][nc-1]
                    if stein and stein.spieler == spieler_id:
                        eigene_nah += 1
        
        score += eigene_nah * 8
        
        return score

    def bewerte_gegner_blockade_nach_bewegung(self, zug, spieler_id):
        """Bewertet Blockadewirkung nach einer Bewegung"""
        score = 0
        to_r, to_c = zug['to']
        gegner_id = 1 if spieler_id == 2 else 2
        
        # Blockiere ich Gegnerzugmöglichkeiten?
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = to_r + dr, to_c + dc
            if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                stein = self.spielfeld_matrix[nr-1][nc-1]
                if stein and stein.spieler == gegner_id:
                    # Prüfe, ob dieser Gegnerstein jetzt blockiert ist
                    if self.ist_gegner_blockiert(nr, nc, gegner_id):
                        score += 25
        
        return score

    def ist_gegner_blockiert(self, r, c, gegner_id):
        """Prüft, ob ein Gegnerstein blockiert ist"""
        # Vereinfachte Prüfung: Hat der Gegnerstein freie orthogonale Felder?
        freie_orthogonale = 0
        for dr, dc in [(0,1), (1,0), (0,-1), (-1,0)]:
            nr, nc = r + dr, c + dc
            if 1 <= nr <= ROWS and 1 <= nc <= COLS:
                if (nr, nc) != self.pfarrer_pos and self.spielfeld_matrix[nr-1][nc-1] is None:
                    freie_orthogonale += 1
        
        return freie_orthogonale == 0

    # ==========================================
    # EINFACHE BEWERTUNGSFUNKTIONEN (für mittlere KI)
    # ==========================================

    def einfache_bewerte_ausrichtung(self, pos, richtung):
        """Einfachere Ausrichtungsbewertung (für mittlere KI)"""
        r, c = pos
        score = 0
        achsen_vektoren = [(0, -1), (0, 1)] if richtung in ['N', 'S'] else [(1, 0), (-1, 0)]
        
        # Nur die ersten 2 Felder in jeder Richtung prüfen
        for dx, dy in achsen_vektoren:
            for dist in [1, 2]:
                nr, nc = r + (dy * dist), c + (dx * dist)
                if (1 <= nr <= ROWS and 1 <= nc <= COLS) and not self.spielfeld_matrix[nr-1][nc-1] and (nr, nc) != self.pfarrer_pos:
                    score += 1
        return score

    def bewerte_ausrichtung(self, pos, richtung):
        """Fortgeschrittene Ausrichtungsbewertung (für schwere KI)"""
        r, c = pos
        score = 0
        achsen_vektoren = [(0, -1), (0, 1)] if richtung in ['N', 'S'] else [(1, 0), (-1, 0)]
        
        for dx, dy in achsen_vektoren:
            dist = 1
            while True:
                nr, nc = r + (dy * dist), c + (dx * dist)
                if not (1 <= nr <= ROWS and 1 <= nc <= COLS) or self.spielfeld_matrix[nr-1][nc-1] or (nr, nc) == self.pfarrer_pos:
                    break
                score += 2  # Jedes freie Feld in der Achse ist gut
                dist += 1
        return score

    # ==========================================
    # INTERAKTION
    # ==========================================

    def handle_click(self, event):
        if not self.spiel_aktiv:
            return
            
        row, col = self.get_grid_coords(event.x, event.y)
        if row is None: return

        valid_moves = self.get_valid_moves(self.aktueller_spieler)
        is_placement_phase = any(m['type'] == 'place' for m in valid_moves)
        stein_in_zelle = self.spielfeld_matrix[row-1][col-1]

        # =========================================================
        # 1️⃣ PFARRER WIRD ANGEKLICKT → PFARRER-MODUS START
        # =========================================================
        if (row, col) == self.pfarrer_pos and not is_placement_phase:
            self.ausgewaehlter_stein_pos = 'pfarrer'
            messagebox.showinfo(
                "Pfarrer",
                "Wähle jetzt einen blockierten Stein zum Tauschen."
            )
            return

        # --- FALL A: PLATZIEREN ---
        if is_placement_phase:
            move_match = None
            for move in valid_moves:
                if move['type'] == 'place' and move['pos'] == (row, col):
                    move_match = move
                    break

            if move_match:
                waehler = RichtungsWaehler(self.master)
                gewaehlte_richtung = waehler.ergebnis
                if gewaehlte_richtung:
                    neues_teil = self.unplatzierte_steine[self.aktueller_spieler].pop(0)
                    neues_teil.ausrichtung = gewaehlte_richtung
                    self.platziere_stein(row, col, neues_teil)
                    if self.pruefe_sieg(self.aktueller_spieler):
                        self.spiel_beenden(f"Spieler {self.aktueller_spieler} hat gewonnen!")
                        return
                    self.wechsle_spieler()
                else: messagebox.showinfo("Info", "Abgebrochen.")
            else:
                self.zeige_platzierungs_fehler(row, col, valid_moves)

        # --- FALL B: BEWEGEN ---
        else:
            # 1.PFARRER-MODUS
            if self.ausgewaehlter_stein_pos == 'pfarrer':
                if self.ist_blockierter_stein(row, col):
                    self.tausche_pfarrer_mit(row, col)
                    self.wechsle_spieler()
                else:
                     messagebox.showwarning(
                        "Ungültig",
                        "Du musst einen blockierten Stein auswählen."
                    )

                         # zurücksetzen!
                self.ausgewaehlter_stein_pos = None
                return
            if not self.ausgewaehlter_stein_pos:
                if stein_in_zelle and stein_in_zelle.spieler == self.aktueller_spieler:
                    kann_ziehen = any(m['type'] == 'move' and m['from'] == (row, col) for m in valid_moves)
                    if kann_ziehen:
                        self.ausgewaehlter_stein_pos = (row, col)
                        self.markiere_stein(row, col, True)
                    else: messagebox.showwarning("Blockiert", "Dieser Stein kann nicht ziehen (Weg versperrt oder Brettende).")
            else:
                start_r, start_c = self.ausgewaehlter_stein_pos
                if (row, col) == (start_r, start_c):
                    self.markiere_stein(start_r, start_c, False)
                    self.ausgewaehlter_stein_pos = None
                    return

                zug_valide = any(m['type'] == 'move' and m['from'] == (start_r, start_c) and m['to'] == (row, col) for m in valid_moves)
                if zug_valide:
                    self.fuehre_zug_aus(row, col)
                    if self.pruefe_sieg(self.aktueller_spieler):
                        self.spiel_beenden(f"Spieler {self.aktueller_spieler} hat gewonnen!")
                        return
                    self.ausgewaehlter_stein_pos = None
                    self.wechsle_spieler()
                else: messagebox.showwarning("Ungültig", "Hierhin kann der Stein nicht ziehen.")

    def zeige_platzierungs_fehler(self, r, c, valid_moves):
        if len(self.unplatzierte_steine[self.aktueller_spieler]) > 0:
            typ = self.unplatzierte_steine[self.aktueller_spieler][0].typ
            if typ in ['Turm', 'Schiff']:
                messagebox.showwarning("Regel", "Kirchenteile müssen auf eine freie Ecke!")
            elif self.spielfeld_matrix[r-1][c-1] is not None: pass
            elif self.hat_eigene_orthogonale_nachbarn(r, c, self.aktueller_spieler):
                messagebox.showwarning("Regel", "Nicht direkt neben eigene Steine bauen!")
            else: messagebox.showinfo("Info", "Ungültiges Feld.")

    def fuehre_zug_aus(self, ziel_row, ziel_col):
        start_row, start_col = self.ausgewaehlter_stein_pos
        stein = self.spielfeld_matrix[start_row-1][start_col-1]

        self.delete(f"piece_{start_row}_{start_col}")
        self.delete(f"arrow_{start_row}_{start_col}")
        self.delete(f"label_{start_row}_{start_col}")

        self.spielfeld_matrix[start_row-1][start_col-1] = None
        self.spielfeld_matrix[ziel_row-1][ziel_col-1] = stein

        stein.drehen()
        self.draw_piece(ziel_row, ziel_col, stein)

    def platziere_stein(self, row, col, stein_obj):
        self.spielfeld_matrix[row-1][col-1] = stein_obj
        self.draw_piece(row, col, stein_obj)

    def wechsle_spieler(self):
        self.aktueller_spieler = 1 if self.aktueller_spieler == 2 else 2
        self.update_title()
        # KI ist dran?
        if self.ki_aktiv and self.aktueller_spieler == self.ki_spieler_id and self.spiel_aktiv:
            self.master.after(500, self.ki_macht_zug)

    def update_title(self):
        vorrat = self.unplatzierte_steine[self.aktueller_spieler]
        if len(vorrat) > 0: phase = f"Platzieren: {vorrat[0].typ}"
        else: phase = "Bewegen"
        ki_info = f" | KI: {self.ki_schwierigkeit}" if self.ki_aktiv else ""
        status = " (Spiel aktiv)" if self.spiel_aktiv else " (Spiel beendet)"
        self.master.title(f"Lass die Kirche im Dorf | Am Zug: {self.spieler_farben[self.aktueller_spieler]} | Phase: {phase}{ki_info}{status}")

    # --- VISUALISIERUNG ---
    def draw_grid(self):
        for i in range(ROWS + 1):
            p = i * CELL_SIZE + OFFSET
            self.create_line(OFFSET, p, WINDOW_SIZE + OFFSET, p, width=LINE_WIDTH, fill="#404040")
            self.create_line(p, OFFSET, p, WINDOW_SIZE + OFFSET, width=LINE_WIDTH, fill="#404040")

    def draw_labels(self):
        for i in range(7):
            c = (i + 0.5) * CELL_SIZE + OFFSET
            self.create_text(c, OFFSET/2, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))
            self.create_text(OFFSET/2, c, text=str(i+1), fill='#333', font=('Arial', 11, 'bold'))

    def get_canvas_coords(self, row, col):
        return (col - 0.5) * CELL_SIZE + OFFSET, (row - 0.5) * CELL_SIZE + OFFSET

    def get_grid_coords(self, x, y):
        if x < OFFSET or y < OFFSET: return None, None
        c, r = int((x-OFFSET)//CELL_SIZE)+1, int((y-OFFSET)//CELL_SIZE)+1
        if 1<=r<=ROWS and 1<=c<=COLS: return r, c
        return None, None

    def draw_pfarrer(self, r, c):
        x, y = self.get_canvas_coords(r, c)
        rad = CELL_SIZE * 0.2
        self.create_oval(x-rad, y-rad, x+rad, y+rad, fill='#333', outline='black', width=2, tags="pfarrer")
        self.create_text(x, y, text="P", fill='white', font=('Arial', 14, 'bold'), tags="pfarrer_label")

    def draw_piece(self, r, c, stein):
        x_c, y_c = self.get_canvas_coords(r, c)
        farbe = stein.get_farbe()
        rad = CELL_SIZE * 0.38

        if stein.typ == 'Haus':
            self.create_rectangle(x_c - rad, y_c - rad, x_c + rad, y_c + rad, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")
        elif stein.typ == 'Schiff':
            self.create_oval(x_c - rad, y_c - rad, x_c + rad, y_c + rad, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")
        elif stein.typ == 'Turm':
            points = [x_c, y_c - rad, x_c + rad, y_c + rad, x_c - rad, y_c + rad]
            self.create_polygon(points, fill=farbe, outline='black', width=2, tags=f"piece_{r}_{c}")

        if stein.typ != 'Pfarrer':
            # VISUALISIERUNG ENTSPRICHT JETZT DER LOGIK: BEIDE RICHTUNGEN
            if stein.ausrichtung in ['N', 'S']:
                dx, dy = 0, 1 # Vertikal
            else:
                dx, dy = 1, 0 # Horizontal

            arrow_len = CELL_SIZE * 0.35
            self.create_line(x_c - dx*arrow_len, y_c - dy*arrow_len, x_c + dx*arrow_len, y_c + dy*arrow_len,
                             arrow=tk.BOTH, width=4, arrowshape=(10, 12, 5), fill='black', tags=f"arrow_{r}_{c}")

            
    def markiere_stein(self, r, c, active):
        if self.spielfeld_matrix[r-1][c-1]:
            outline = '#FFFF00' if active else 'black'
            width = 5 if active else 2
            self.itemconfig(f"piece_{r}_{c}", outline=outline, width=width)

if __name__ == '__main__':
    root = tk.Tk()
    root.title("Lass die Kirche im Dorf")
    root.geometry(f"{WINDOW_SIZE + 2*OFFSET + 20}x{WINDOW_SIZE + 2*OFFSET + 20}")
    sb = Spielbrett(root)
    root.mainloop()